<a href="https://colab.research.google.com/github/UjalaJha/nova/blob/develop/notebook/feature_vector_content_based_filtering_description_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#import libraries
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
import ast

In [103]:
# !pip install langdetect
# !pip install ntlk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [4]:
# Mount Google Drive folder
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Path to file in mounted Google Drive folder
file_path = '/content/drive/MyDrive/Grace Hopper/combined-1kAbove-stars.csv'

# Read CSV file using pandas
data = pd.read_csv(file_path)
data.head()

,Unnamed: 0.1,Unnamed: 0,id,node_id,name,full_name,private,owner,html_url,description,...,visibility,forks,open_issues,watchers,default_branch,permissions,score,lang_name,lang_perc,license_name
0,0,0,45717250,MDEwOlJlcG9zaXRvcnk0NTcxNzI1MA==,tensorflow,tensorflow/tensorflow,False,"{'login': 'tensorflow', 'id': 15658638, 'node_...",https://github.com/tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,...,public,88115,2216,173348,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['C++', 'Python', 'MLIR', 'Starlark', 'HTML', ...","[132095537, 43756548, 13084966, 8623743, 46864...",Apache-2.0
1,1,1,20580498,MDEwOlJlcG9zaXRvcnkyMDU4MDQ5OA==,kubernetes,kubernetes/kubernetes,False,"{'login': 'kubernetes', 'id': 13629408, 'node_...",https://github.com/kubernetes/kubernetes,Production-Grade Container Scheduling and Mana...,...,public,35807,2486,97468,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['Go', 'Shell', 'PowerShell', 'Makefile', 'Doc...","[66222388, 1836554, 144474, 63695, 49417, 2373...",Apache-2.0
2,2,2,155220641,MDEwOlJlcG9zaXRvcnkxNTUyMjA2NDE=,transformers,huggingface/transformers,False,"{'login': 'huggingface', 'id': 25720743, 'node...",https://github.com/huggingface/transformers,🤗 Transformers: State-of-the-art Machine Learn...,...,public,19596,672,93079,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['Python', 'Cuda', 'Shell', 'Dockerfile', 'C++...","[42853265, 175419, 30374, 20516, 12959, 6021, ...",Apache-2.0
3,3,3,20929025,MDEwOlJlcG9zaXRvcnkyMDkyOTAyNQ==,TypeScript,microsoft/TypeScript,False,"{'login': 'microsoft', 'id': 6154722, 'node_id...",https://github.com/microsoft/TypeScript,TypeScript is a superset of JavaScript that co...,...,public,11681,6108,90382,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['TypeScript', 'JavaScript', 'Dockerfile', 'Sh...","[87866426, 198563, 330, 53]",Apache-2.0
4,4,4,90796663,MDEwOlJlcG9zaXRvcnk5MDc5NjY2Mw==,puppeteer,puppeteer/puppeteer,False,"{'login': 'puppeteer', 'id': 6906516, 'node_id...",https://github.com/puppeteer/puppeteer,Headless Chrome Node.js API,...,public,8877,268,82712,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0,"['TypeScript', 'JavaScript', 'HTML', 'CSS', 'D...","[1859434, 112872, 25585, 6581, 1574, 600]",Apache-2.0


In [ ]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'node_id', 'name', 'full_name',
       'private', 'owner', 'html_url', 'description', 'fork', 'url',
       'forks_url', 'keys_url', 'collaborators_url', 'teams_url', 'hooks_url',
       'issue_events_url', 'events_url', 'assignees_url', 'branches_url',
       'tags_url', 'blobs_url', 'git_tags_url', 'git_refs_url', 'trees_url',
       'statuses_url', 'languages_url', 'stargazers_url', 'contributors_url',
       'subscribers_url', 'subscription_url', 'commits_url', 'git_commits_url',
       'comments_url', 'issue_comment_url', 'contents_url', 'compare_url',
       'merges_url', 'archive_url', 'downloads_url', 'issues_url', 'pulls_url',
       'milestones_url', 'notifications_url', 'labels_url', 'releases_url',
       'deployments_url', 'created_at', 'updated_at', 'pushed_at', 'git_url',
       'ssh_url', 'clone_url', 'svn_url', 'homepage', 'size',
       'stargazers_count', 'watchers_count', 'language', 'has_issues',
       'has_projects', 'ha

# Preprocess Data

In [6]:
# Data Picking and Analysis
from IPython.display import display
data_selected = data[["full_name","name","description","topics","language","lang_name","license_name"]]
print('\n\nAll Data -------------------------------------------------------------\n\n')
display(data_selected)
print('\n\nAll Data HEAD -------------------------------------------------------------\n\n')
display(data_selected.head())
print('\n\nAll Data Describe -------------------------------------------------------------\n\n')
display(data_selected.describe())
print('\n\nAll Data COUNT -------------------------------------------------------------\n\n')
display(data_selected.count())



All Data -------------------------------------------------------------




,full_name,name,description,topics,language,lang_name,license_name
0,tensorflow/tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,"['deep-learning', 'deep-neural-networks', 'dis...",C++,"['C++', 'Python', 'MLIR', 'Starlark', 'HTML', ...",Apache-2.0
1,kubernetes/kubernetes,kubernetes,Production-Grade Container Scheduling and Mana...,"['cncf', 'containers', 'go', 'kubernetes']",Go,"['Go', 'Shell', 'PowerShell', 'Makefile', 'Doc...",Apache-2.0
2,huggingface/transformers,transformers,🤗 Transformers: State-of-the-art Machine Learn...,"['bert', 'deep-learning', 'flax', 'hacktoberfe...",Python,"['Python', 'Cuda', 'Shell', 'Dockerfile', 'C++...",Apache-2.0
3,microsoft/TypeScript,TypeScript,TypeScript is a superset of JavaScript that co...,"['javascript', 'language', 'typechecker', 'typ...",TypeScript,"['TypeScript', 'JavaScript', 'Dockerfile', 'Sh...",Apache-2.0
4,puppeteer/puppeteer,puppeteer,Headless Chrome Node.js API,"['automation', 'chrome', 'chromium', 'develope...",TypeScript,"['TypeScript', 'JavaScript', 'HTML', 'CSS', 'D...",Apache-2.0
...,...,...,...,...,...,...,...
5320,hashicorp/setup-terraform,setup-terraform,Sets up Terraform CLI in your GitHub Actions w...,[],JavaScript,"['JavaScript', 'Shell']",MPL-2.0
5321,luguanxing/Cheating-Plugin-Program,Cheating-Plugin-Program,从零开始研究外挂设计原理,[],C++,"['C++', 'C', 'Makefile']",MPL-2.0
5322,actiontech/sqle,sqle,SQLE is a SQL audit platform | SQLE 是一个支持多场景，原...,"['advisor', 'audit', 'jetbrains', 'mssql', 'my...",Go,"['Go', 'Makefile', 'Shell', 'Dockerfile', 'Pyt...",MPL-2.0
5323,gzuidhof/starboard-notebook,starboard-notebook,In-browser literate notebooks,"['literate-programming', 'notebook', 'starboar...",TypeScript,"['TypeScript', 'SCSS', 'JavaScript', 'CSS', 'M...",MPL-2.0




All Data HEAD -------------------------------------------------------------




,full_name,name,description,topics,language,lang_name,license_name
0,tensorflow/tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,"['deep-learning', 'deep-neural-networks', 'dis...",C++,"['C++', 'Python', 'MLIR', 'Starlark', 'HTML', ...",Apache-2.0
1,kubernetes/kubernetes,kubernetes,Production-Grade Container Scheduling and Mana...,"['cncf', 'containers', 'go', 'kubernetes']",Go,"['Go', 'Shell', 'PowerShell', 'Makefile', 'Doc...",Apache-2.0
2,huggingface/transformers,transformers,🤗 Transformers: State-of-the-art Machine Learn...,"['bert', 'deep-learning', 'flax', 'hacktoberfe...",Python,"['Python', 'Cuda', 'Shell', 'Dockerfile', 'C++...",Apache-2.0
3,microsoft/TypeScript,TypeScript,TypeScript is a superset of JavaScript that co...,"['javascript', 'language', 'typechecker', 'typ...",TypeScript,"['TypeScript', 'JavaScript', 'Dockerfile', 'Sh...",Apache-2.0
4,puppeteer/puppeteer,puppeteer,Headless Chrome Node.js API,"['automation', 'chrome', 'chromium', 'develope...",TypeScript,"['TypeScript', 'JavaScript', 'HTML', 'CSS', 'D...",Apache-2.0




All Data Describe -------------------------------------------------------------




,full_name,name,description,topics,language,lang_name,license_name
count,5325,5324,5280,5325,4945,5325,5325
unique,5277,5175,5229,3839,100,3129,9
top,redux-form/redux-form,core,A data-driven UICollectionView framework for b...,[],Python,[],Apache-2.0
freq,2,5,2,1424,902,366,1000




All Data COUNT -------------------------------------------------------------




full_name       5325
name            5324
description     5280
topics          5325
language        4945
lang_name       5325
license_name    5325
dtype: int64

In [ ]:
print('\n\nUnique Counts -------------------------------------------------------------\n\n')
display(data_selected["language"].unique())
display(len(data_selected["language"].unique()))

display(data_selected["license_name"].unique())
display(len(data_selected["license_name"].unique()))




Unique Counts -------------------------------------------------------------




array(['C++', 'Go', 'Python', 'TypeScript', 'C', 'Java', 'Rust',
       'Dockerfile', nan, 'Kotlin', 'JavaScript', 'Shell', 'Scala',
       'SCSS', 'Lua', 'HTML', 'Jupyter Notebook', 'Swift', 'Elixir',
       'Vue', 'Crystal', 'Makefile', 'Objective-C', 'Mustache', 'VimL',
       'WebAssembly', 'Starlark', 'Groovy', 'Markdown', 'Dart', 'Jinja',
       'PHP', 'Pascal', 'Ruby', 'Erlang', 'C#', 'PowerShell', 'Cython',
       'CSS', 'Assembly', 'Svelte', 'Prolog', 'Scheme', 'Jsonnet',
       'PLpgSQL', 'Handlebars', 'Vim Script', 'Emacs Lisp', 'Perl',
       'Matlab', 'Nim', 'TeX', 'ASL', 'CMake', 'Factor', 'Haskell',
       'YARA', 'MATLAB', 'CoffeeScript', 'Elm', 'Dhall', 'Odin',
       'AppleScript', 'Batchfile', 'Vim script', 'PostScript', 'Pawn',
       'Verilog', 'Cuda', 'Solidity', 'SVG', 'DIGITAL Command Language',
       'JSON', 'R', 'Clojure', 'Objective-C++', 'Vala', 'D', 'XSLT',
       'Mercury', 'HLSL', 'Smali', 'Visual Basic .NET', 'AutoHotkey',
       'Blade', 'OCaml', 'Haxe

101

array(['Apache-2.0', 'BSD-2-Clause', 'BSD-3-Clause', 'CC0-1.0', 'EPL-2.0',
       'GPL-3.0', 'LGPL-3.0', 'MIT', 'MPL-2.0'], dtype=object)

9

In [ ]:
#This is funny the Repo is named NULL and hence its name is detected in NA LOL
df_name_na=data_selected[data_selected["name"].isna()]
display(df_name_na)

,full_name,name,description,stargazers_count,forks_count,topics,language,lang_name,license_name
1270,guregu/null,NaN,reasonable handling of nullable values,1546,224,"['go', 'json', 'sql']",Go,['Go'],BSD-2-Clause


In [ ]:
# Case where topic is empty
result = data_selected[data_selected['lang_name'].str.contains('Adblock')]
display(result)

,full_name,name,description,stargazers_count,forks_count,topics,language,lang_name,license_name
3688,uBlockOrigin/uAssets,uAssets,"Resources for uBlock Origin, uMatrix: static f...",2427,561,[],Adblock Filter List,"['Adblock Filter List', 'JavaScript', 'Makefil...",GPL-3.0
3969,cjx82630/cjxlist,cjxlist,NaN,1425,129,[],Adblock Filter List,['Adblock Filter List'],LGPL-3.0


In [212]:
repo_data = data_selected.copy()
display(repo_data.head())

,full_name,name,description,topics,language,lang_name,license_name
0,tensorflow/tensorflow,tensorflow,An Open Source Machine Learning Framework for Everyone,"['deep-learning', 'deep-neural-networks', 'distributed', 'machine-learning', 'ml', 'neural-network', 'python', 'tensorflow']",C++,"['C++', 'Python', 'MLIR', 'Starlark', 'HTML', 'Go', 'C', 'Java', 'Jupyter Notebook', 'Shell', 'Objective-C++', 'CMake', 'Objective-C', 'Smarty', 'Swift', 'Batchfile', 'SourcePawn', 'C#', 'Ruby', 'Perl', 'LLVM', 'Pawn', 'Dockerfile', 'Cython', 'Roff', 'Makefile', 'Vim Snippet']",Apache-2.0
1,kubernetes/kubernetes,kubernetes,Production-Grade Container Scheduling and Management,"['cncf', 'containers', 'go', 'kubernetes']",Go,"['Go', 'Shell', 'PowerShell', 'Makefile', 'Dockerfile', 'Python', 'C', 'sed', 'Batchfile', 'HTML']",Apache-2.0
2,huggingface/transformers,transformers,"🤗 Transformers: State-of-the-art Machine Learning for Pytorch, TensorFlow, and JAX.","['bert', 'deep-learning', 'flax', 'hacktoberfest', 'jax', 'language-model', 'language-models', 'machine-learning', 'model-hub', 'natural-language-processing', 'nlp', 'nlp-library', 'pretrained-models', 'python', 'pytorch', 'pytorch-transformers', 'seq2seq', 'speech-recognition', 'tensorflow', 'transformer']",Python,"['Python', 'Cuda', 'Shell', 'Dockerfile', 'C++', 'C', 'Cython', 'Makefile', 'Jsonnet']",Apache-2.0
3,microsoft/TypeScript,TypeScript,TypeScript is a superset of JavaScript that compiles to clean JavaScript output.,"['javascript', 'language', 'typechecker', 'typescript']",TypeScript,"['TypeScript', 'JavaScript', 'Dockerfile', 'Shell']",Apache-2.0
4,puppeteer/puppeteer,puppeteer,Headless Chrome Node.js API,"['automation', 'chrome', 'chromium', 'developer-tools', 'headless-chrome', 'node-module', 'testing', 'web']",TypeScript,"['TypeScript', 'JavaScript', 'HTML', 'CSS', 'Dockerfile', 'Shell']",Apache-2.0


In [213]:
#converting into list and replacing space by hyphen for each data
repo_data['topics'] = repo_data['topics'].apply(lambda x: ast.literal_eval(x))
repo_data['topics'] = repo_data['topics'].apply(lambda x: [i.replace(' ', '-') for i in x])


repo_data['lang_name'] = repo_data['lang_name'].apply(lambda x: ast.literal_eval(x))
repo_data['lang_name'] = repo_data['lang_name'].apply(lambda x: [i.replace(' ', '-') for i in x])

repo_data['language'] = repo_data['language'].astype(str)
repo_data['language'] = repo_data['language'].apply(lambda x: x.replace(' ', '-'))


In [214]:
#Unique language set
langs_tokens = repo_data["lang_name"]
lang_tokens_list = list()
lang_tokens_set = set()
for token in langs_tokens:
  lang_tokens_list.append(token)
  lang_tokens_set = lang_tokens_set.union(set(token))

display(len(lang_tokens_list))
display(len(lang_tokens_set))
display(repo_data['language'].unique())

5325

372

array(['C++', 'Go', 'Python', 'TypeScript', 'C', 'Java', 'Rust',
       'Dockerfile', 'nan', 'Kotlin', 'JavaScript', 'Shell', 'Scala',
       'SCSS', 'Lua', 'HTML', 'Jupyter-Notebook', 'Swift', 'Elixir',
       'Vue', 'Crystal', 'Makefile', 'Objective-C', 'Mustache', 'VimL',
       'WebAssembly', 'Starlark', 'Groovy', 'Markdown', 'Dart', 'Jinja',
       'PHP', 'Pascal', 'Ruby', 'Erlang', 'C#', 'PowerShell', 'Cython',
       'CSS', 'Assembly', 'Svelte', 'Prolog', 'Scheme', 'Jsonnet',
       'PLpgSQL', 'Handlebars', 'Vim-Script', 'Emacs-Lisp', 'Perl',
       'Matlab', 'Nim', 'TeX', 'ASL', 'CMake', 'Factor', 'Haskell',
       'YARA', 'MATLAB', 'CoffeeScript', 'Elm', 'Dhall', 'Odin',
       'AppleScript', 'Batchfile', 'Vim-script', 'PostScript', 'Pawn',
       'Verilog', 'Cuda', 'Solidity', 'SVG', 'DIGITAL-Command-Language',
       'JSON', 'R', 'Clojure', 'Objective-C++', 'Vala', 'D', 'XSLT',
       'Mercury', 'HLSL', 'Smali', 'Visual-Basic-.NET', 'AutoHotkey',
       'Blade', 'OCaml', 'Ha

In [215]:
#Unique topic set
topics_tokens = repo_data["topics"]
topics_tokens_list = list()
topics_tokens_set = set()
for token in topics_tokens:
  topics_tokens_list.append(token)
  topics_tokens_set = topics_tokens_set.union(set(token))

display(len(topics_tokens_list))
display(len(topics_tokens_set))

5325

11567

In [216]:
# Step 1: Tokenize the description field into individual words or tokens
import re

def tokenize_text(text):
    # Return None if text is empty
    if not text:
        return []
    if isinstance(text, float):
      return []
    if isinstance(text, int):
      return []
    # Use regex to tokenize the text
    tokens = re.findall('\w+', text.lower())
    return tokens

repo_data['description_tokens'] = repo_data['description'].apply(tokenize_text)
repo_data['description_tokens'] 

0                                    [an, open, source, machine, learning, framework, for, everyone]
1                                        [production, grade, container, scheduling, and, management]
2         [transformers, state, of, the, art, machine, learning, for, pytorch, tensorflow, and, jax]
3       [typescript, is, a, superset, of, javascript, that, compiles, to, clean, javascript, output]
4                                                                  [headless, chrome, node, js, api]
                                                    ...                                             
5320                                 [sets, up, terraform, cli, in, your, github, actions, workflow]
5321                                                                                  [从零开始研究外挂设计原理]
5322       [sqle, is, a, sql, audit, platform, sqle, 是一个支持多场景, 原生支持, mysql, 审核且数据库类型可扩展的, sql, 审核工具]
5323                                                              [in, browser, literate, n

In [217]:
# Step 2: Remove punctions and number from this
import nltk
nltk.download('stopwords')
from langdetect import detect
from nltk.corpus import stopwords
import string
punctuations = set(string.punctuation)
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    # Remove stop words from the token list
    filtered_tokens = []
    for token in tokens:
        # Check if token is alphabetic, not a stop word, not a number or float, not a punctuation, and is ASCII
        if token.isalpha() and token.lower() not in stop_words and not token.isdigit() and not token.replace('.','',1).isdigit() and token not in punctuations and token.isascii():
            filtered_tokens.append(token)
    return filtered_tokens


repo_data['description_tokens'] = repo_data['description_tokens'].apply(remove_stopwords)
repo_data['description_tokens']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0                        [open, source, machine, learning, framework, everyone]
1                        [production, grade, container, scheduling, management]
2       [transformers, state, art, machine, learning, pytorch, tensorflow, jax]
3       [typescript, superset, javascript, compiles, clean, javascript, output]
4                                             [headless, chrome, node, js, api]
                                         ...                                   
5320                          [sets, terraform, cli, github, actions, workflow]
5321                                                                         []
5322                             [sqle, sql, audit, platform, sqle, mysql, sql]
5323                                             [browser, literate, notebooks]
5324                              [automated, api, testing, quality, assurance]
Name: description_tokens, Length: 5325, dtype: object

In [218]:
# Step 3: Retaining only language and topic tokens

def remove_tokens_irrelevant(tokens):
    filtered_tokens = []
    for token in tokens:
        if token in lang_tokens_set or token in topics_tokens_set:
            filtered_tokens.append(token)
    return filtered_tokens

repo_data['description_tokens'] = repo_data['description_tokens'].apply(remove_tokens_irrelevant)
repo_data['description_tokens']

0                                  [open, source, machine, learning, framework]
1                               [production, container, scheduling, management]
2       [transformers, state, art, machine, learning, pytorch, tensorflow, jax]
3                                [typescript, superset, javascript, javascript]
4                                             [headless, chrome, node, js, api]
                                         ...                                   
5320                                [terraform, cli, github, actions, workflow]
5321                                                                         []
5322                                         [sql, audit, platform, mysql, sql]
5323                                                       [browser, notebooks]
5324                                                    [api, testing, quality]
Name: description_tokens, Length: 5325, dtype: object

In [ ]:
# TESTING
result = 'art' in lang_tokens_set
result2 = 'art' in topics_tokens_set
display(result)
display(result2)
print(topics_tokens_set)

In [122]:

# # NOT needed as we are doing ascii checks
# # Step 3: Remove all non english,special character,non ascii tokens
# from langdetect import detect
# from langdetect.lang_detect_exception import LangDetectException

# def remove_non_english(tokens):
#     filtered_tokens = []
#     for token in tokens:
#         try:
#             if token.isalpha() and detect(token) == 'en':
#                 filtered_tokens.append(token)
#         except LangDetectException:
#             print(token)
#     return filtered_tokens

# # repo_data['description_tokens'].count_na()
# if not repo_data['description_tokens'].isna().all():
#     repo_data['description_tokens'] = repo_data['description_tokens'].apply(remove_non_english)

# # repo_data['description_tokens']

In [ ]:
# # Output is not as expected 
# # Apply stemming to the remaining tokens to reduce them to their base form
# from nltk.stem.porter import PorterStemmer

# stemmer = PorterStemmer()

# def stem_tokens(tokens):
#     # Apply stemming to the token list
#     stemmed_tokens = [stemmer.stem(token) for token in tokens]
#     return stemmed_tokens

# repo_data['description_tokens'] = repo_data['description_tokens'].apply(stem_tokens)
# repo_data['description_tokens'].to_string()

In [220]:
#Unique topic set
import ast
tokens = repo_data["description_tokens"]
tokens_list = list()
tokens_set = set()
for token in tokens:
  tokens_list.append(token)
  tokens_set = tokens_set.union(set(token))

display(len(tokens_list))
display(len(tokens_set))

5325

3002

In [221]:
# Joining all data in list by space and converting lower case
repo_data['topics'] = repo_data['topics'].apply(lambda x: ' '.join(x))
repo_data['topics'] = repo_data['topics'].apply(lambda x: x.lower())

repo_data['lang_name'] = repo_data['lang_name'].apply(lambda x: ' '.join(x))
repo_data['lang_name'] = repo_data['lang_name'].apply(lambda x: x.lower())

repo_data['description_tokens'] = repo_data['description_tokens'].apply(lambda x: ' '.join(x))
repo_data['description_tokens'] = repo_data['description_tokens'].apply(lambda x: x.lower())

repo_data['language'] = repo_data['language'].astype(str).apply(lambda x: x.lower())
repo_data['name'] = repo_data['name'].astype(str).apply(lambda x: x.lower())
repo_data['license_name'] = repo_data['license_name'].astype(str).apply(lambda x: x.lower())

In [222]:
display(repo_data.head())

,full_name,name,description,topics,language,lang_name,license_name,description_tokens
0,tensorflow/tensorflow,tensorflow,An Open Source Machine Learning Framework for Everyone,deep-learning deep-neural-networks distributed machine-learning ml neural-network python tensorflow,c++,c++ python mlir starlark html go c java jupyter-notebook shell objective-c++ cmake objective-c smarty swift batchfile sourcepawn c# ruby perl llvm pawn dockerfile cython roff makefile vim-snippet,apache-2.0,open source machine learning framework
1,kubernetes/kubernetes,kubernetes,Production-Grade Container Scheduling and Management,cncf containers go kubernetes,go,go shell powershell makefile dockerfile python c sed batchfile html,apache-2.0,production container scheduling management
2,huggingface/transformers,transformers,"🤗 Transformers: State-of-the-art Machine Learning for Pytorch, TensorFlow, and JAX.",bert deep-learning flax hacktoberfest jax language-model language-models machine-learning model-hub natural-language-processing nlp nlp-library pretrained-models python pytorch pytorch-transformers seq2seq speech-recognition tensorflow transformer,python,python cuda shell dockerfile c++ c cython makefile jsonnet,apache-2.0,transformers state art machine learning pytorch tensorflow jax
3,microsoft/TypeScript,typescript,TypeScript is a superset of JavaScript that compiles to clean JavaScript output.,javascript language typechecker typescript,typescript,typescript javascript dockerfile shell,apache-2.0,typescript superset javascript javascript
4,puppeteer/puppeteer,puppeteer,Headless Chrome Node.js API,automation chrome chromium developer-tools headless-chrome node-module testing web,typescript,typescript javascript html css dockerfile shell,apache-2.0,headless chrome node js api


In [223]:
# creating a feature vector
repo_data['feature_vector'] =  repo_data[['topics','language','description_tokens']].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
display(repo_data["feature_vector"])

0                                                                                                                                                                                      deep-learning deep-neural-networks distributed machine-learning ml neural-network python tensorflow c++ open source machine learning framework
1                                                                                                                                                                                                                                                         cncf containers go kubernetes go production container scheduling management
2       bert deep-learning flax hacktoberfest jax language-model language-models machine-learning model-hub natural-language-processing nlp nlp-library pretrained-models python pytorch pytorch-transformers seq2seq speech-recognition tensorflow transformer python transformers state art machine learning pytorch tensorflow jax
3                     

# Extract features

In [224]:
recomendation_df=repo_data[['full_name','feature_vector']]
display(recomendation_df[recomendation_df['full_name'].str.contains('uBlockOrigin/uAssets')])
display(recomendation_df[recomendation_df['full_name'].str.contains('uBlockOrigin/uAssets')])

,full_name,feature_vector
3688,uBlockOrigin/uAssets,adblock-filter-list resources ublock static filter lists


,full_name,feature_vector
3688,uBlockOrigin/uAssets,adblock-filter-list resources ublock static filter lists


In [225]:
# Import necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(recomendation_df['feature_vector'])
display(feature_matrix.get_shape())

(5325, 7978)

In [46]:
# # Import necessary libraries
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # Create a TfidfVectorizer to tokenize the features string and create a sparse matrix
# feature_matrix = TfidfVectorizer()
# feature_matrix = feature_matrix.fit_transform(repo_data['feature_vector'])
# display(feature_matrix.get_shape())

(5325, 11722)

# Compute similarity

In [226]:
feature_similarity = cosine_similarity(feature_matrix)

In [227]:
display(feature_similarity)

array([[1.        , 0.        , 0.40325254, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.09534626, 0.        ,
        0.        ],
       [0.40325254, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.09534626, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.03120432],
       [0.        , 0.        , 0.        , ..., 0.        , 0.03120432,
        1.        ]])

In [228]:
feature_similarity.shape

(5325, 5325)

# Recommendation Engine

In [229]:
# set the display option to show full column width
pd.set_option('display.max_colwidth', None)

def recommend_repo(features, n=10):
    # Find repos with similar features
    features_list = features.split(',')
    

    # Find repos with similar features
    feature_indices = set()
    # for feature in features_list:
        # feature_indices.update(recomendation_df[recomendation_df['feature_vector'].str.contains(feature)].index)
    feature_indices.update(recomendation_df[recomendation_df['feature_vector'].apply(lambda x: any(feature in x.split() for feature in features_list))].index)
    # print(recomendation_df.iloc[list(feature_indices)]["feature_vector"].to_string())
    # display(recomendation_df[recomendation_df['feature_vector'].apply(lambda x: any(feature in x.split() for feature in features_list))])
    
    # Compute repo similarity scores
    similarity_scores = []
    for repo in feature_indices:
        feature_similarities = feature_similarity[repo, list(feature_indices)]
        # print(repo_data.iloc[[repo]["full_name"]])
        similarity_scores.append((repo, feature_similarities.mean()))
    
    # Sort by similarity scores and recommend top N repos
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:n]
    recommended_repo = [repo_data.iloc[score[0]]['full_name'] for score in similarity_scores]
    
    return recommended_repo


In [230]:
# set the display option to show full column width
pd.set_option('display.max_colwidth', None)
recommend_repo('spring,java,spring-boot',n=5)

['proxyee-down-org/proxyee-down',
 'bilibili/DanmakuFlameMaster',
 'alibaba/otter',
 'hongyangAndroid/okhttputils',
 'hongyangAndroid/AndroidAutoLayout']

In [231]:
recommend_repo('javascript,language,typechecker,compile,output,typescript',n=5)

['EastWorld/wechat-app-mall',
 'kesenhoo/android-training-course-in-chinese',
 'yemount/pose-animator',
 'fex-team/fis3',
 'fex-team/kityminder']

In [232]:
recommend_repo('testing,chrome,automation,web,chromium,developer-tools,node-module,headless-chrome,typescript,javascript')

['EastWorld/wechat-app-mall',
 'kesenhoo/android-training-course-in-chinese',
 'yemount/pose-animator',
 'fex-team/fis3',
 'fex-team/kityminder',
 'hughsk/flat',
 'fex-team/ua-device',
 'abourget/gevent-socketio',
 'fex-team/kityminder-core',
 'mapbox/mapbox-studio-classic']

In [233]:
recommend_repo('kubernetes,go,containers,dockerfile,sed,batchfile,cncf')

['tarm/serial',
 'Go-zh/tour',
 'studygolang/studygolang',
 'kardianos/govendor',
 'cloudflare/tableflip',
 'google/starlark-go',
 'spf13/viper',
 'glycerine/zygomys',
 'fsnotify/fsnotify',
 'golangci/golangci-lint']

In [234]:
recommend_repo('adblock-filter-list',n=5)

['cjx82630/cjxlist', 'uBlockOrigin/uAssets']

In [235]:
recommend_repo('laravel,php',n=5)

['zendframework/zend-code',
 'iagox86/dnscat2',
 'sebastianbergmann/recursion-context',
 'sebastianbergmann/comparator',
 'yiisoft/yii']